In [1]:
data_path = '/home/oscar/Desktop/Exjobb/Data/ieee_supplement/Hyperspectral_Grids/Salinas/'

In [2]:
import numpy as np
import visdom
import utils
import seaborn as sns
from datasets import get_dataset, HyperX
import os
from sklearn import preprocessing
from sklearn.decomposition import PCA
import random

In [4]:
vis = visdom.Visdom()

Setting up a new session...


In [5]:
label_values = ["Undefined", "Brocoli_green_weeds_1", "Brocoli_green_weeds_2", "Fallow",
                        "Fallow_rough_plow", "Fallow_smooth", "Stubble",
                        "Celery", "Grapes_untrained", "Soil_vinyard_develop",
                        "Corn_senesced_green_weeds", "Lettuce_romaine_4wk", "Lettuce_romaine_5wk",
                        "Lettuce_romaine_6wk", "Lettuce_romaine_7wk", "Vinyard_untrained",
                        "Vinyard_vertical_trellis"]

palette = None
rgb_bands = (43, 21, 11)

if palette is None:
    # Generate color palette
    palette = {0: (0, 0, 0)}
    for k, color in enumerate(sns.color_palette("hls", len(label_values) - 1)):
        palette[k + 1] = tuple(np.asarray(255 * np.array(color), dtype='uint8'))
invert_palette = {v: k for k, v in palette.items()}

def convert_to_color(x):
    return utils.convert_to_color_(x, palette=palette)
def convert_from_color(x):
    return utils.convert_from_color_(x, palette=invert_palette)

In [87]:
#train_patches = np.zeros(int((len(os.listdir(data_path + '/salinas_fold_0/'))-3)/2))
#train_gt = np.copy(train_patches)

train_patches = []
train_gt = []

for i in (range(int((len(os.listdir(data_path + '/salinas_fold_0/'))-3)/2))):
    #train_patches[i] = np.load(data_path + '/salinas_fold_0/patch_{}.npy'.format(i))
    #train_gt[i] = np.load(data_path + '/salinas_fold_0/patch_{}_gt.npy'.format(i))
    
    train_patches.append(np.load(data_path + '/salinas_fold_0/patch_{}.npy'.format(i)))
    train_gt.append(np.load(data_path + '/salinas_fold_0/patch_{}_gt.npy'.format(i)))

test = np.load(data_path + '/salinas_fold_0/test.npy')
test_gt = np.load(data_path + '/salinas_fold_0/test_gt.npy') 

In [ ]:
mask = np.ones_like(train_gt)
print(np.count_nonzero(mask))
for i in range(len(mask)):
    mask[i, train_gt[i] == 0] = 0
print(np.count_nonzero(mask))
p,x,y = np.nonzero(mask)
print(str(p) + ' ' + str(x) + '' + str(y))
p_u, x_u, y_u = np.nonzero(mask==0)

In [88]:
gt_np = np.array(train_gt)
train_np = np.array(train_patches)

In [ ]:
idx = set()

while len(idx) < 0.05*np.prod(gt_np.shape):
    p = np.random.randint(gt_np.shape[0])
    x = np.random.randint(gt_np.shape[1])
    y = np.random.randint(gt_np.shape[2])
    if (p,x,y) not in idx:
        idx.add((p,x,y))

In [ ]:
idx_np = np.array(list(idx))
idx_np

In [ ]:
train_np.shape

In [89]:
ignored_labels = [0]
patch_size = 5

pad_width = patch_size // 2

train_np = np.pad(train_np, ((0,0), (pad_width, pad_width), (pad_width, pad_width), (0,0)))
gt_np = np.pad(gt_np, ((0,0), (pad_width, pad_width), (pad_width, pad_width)))

In [91]:
mask = np.ones_like(gt_np)
for l in ignored_labels:
    mask[gt_np == l] = 0
patch_labeled, x_labeled, y_labeled = np.nonzero(mask)

for l in ignored_labels:
    patch_unlabeled, x_unlabeled, y_unlabeled = np.nonzero(mask==l)

p = patch_size // 2

indices_labeled = np.array([(p_l, x_l, y_l) for p_l, x_l, y_l in zip(patch_labeled, x_labeled, y_labeled) if x_l >= p and x_l < train_np.shape[1] - p and y_l >= p and y_l < train_np.shape[2] - p])
labels = [gt_np[p_l, x_l, y_l] for p_l, x_l, y_l in indices_labeled]

indices_unlabeled = np.array([(p_u, x_u, y_u) for p_u, x_u, y_u in zip(patch_unlabeled, x_unlabeled, y_unlabeled) if x_u >= p and x_u < train_np.shape[1] - p and y_u >= p and y_u < train_np.shape[2] - p])

indices_labeled_shuffle = np.copy(indices_labeled)
indices_unlabeled_shuffle = np.copy(indices_unlabeled)

np.random.shuffle(indices_labeled_shuffle)
np.random.shuffle(indices_unlabeled_shuffle)

In [ ]:
p, x, y = indices_labeled[0]

data = train_np[p, x-pad_width:x - pad_width + patch_size, y-pad_width:y-pad_width+patch_size]

label = gt_np[p, x-pad_width:x - pad_width + patch_size, y-pad_width:y-pad_width+patch_size]
labels[0]

In [ ]:
reshape = train_np.reshape(np.prod(train_np.shape[:3]), np.prod(train_np.shape[3:]))
scaler = preprocessing.MinMaxScaler()
reshape = scaler.fit_transform(reshape)
img_list = reshape.reshape(train_np.shape)

test = np.asarray(test, dtype='float32')
data_test = test.reshape(np.prod(test.shape[:2]), np.prod(test.shape[2:]))
data_test = scaler.transform(data_test)
test_patch = data_test.reshape(test.shape)

In [ ]:
np.max((test - np.min(train_np))/(np.max(train_np) - np.min(train_np)))

In [93]:
x_size = train_np[0].shape[0]
y_size = train_np[0].shape[1]
data_train = np.array([train_np[p_l, x_l, y_l] for p_l, x_l, y_l in zip(patch_labeled, x_labeled, y_labeled) if x_l >= p and x_l < x_size - p and y_l >= p and y_l < y_size - p])

    
    

In [ ]:
pca = PCA(n_components=11)
pca.fit(data_train)

In [ ]:
nalepa_classes = np.zeros(len(label_values))

for i in range(len(train_gt)):
    for c in range(len(nalepa_classes)):
        amount = np.count_nonzero(train_gt[i] == c)
        nalepa_classes[c] += amount

In [ ]:
print(nalepa_classes)

In [ ]:
print(np.sum(nalepa_classes[1:]))

In [ ]:
np.count_nonzero(gt_np) + np.count_nonzero(test_gt)

In [ ]:
np.shape(np.pad(train_patches, ((0,0), (2,2), (2,2), (0,0))))

In [ ]:
utils.display_predictions(convert_to_color(gt_5), vis)

In [ ]:
utils.display_dataset(patch_5, gt_5, rgb_bands, label_values, palette, vis)

In [5]:
img, gt, label_values, ignored_labels, rgb_bands, palette = get_dataset('Salinas', target_folder='/home/oscar/Desktop/Exjobb/Data/')

In [ ]:
img.shape[1]

In [ ]:
patch_size = 5

In [ ]:
x = 0
y = 0

patch_list = []

while y in range(img.shape[1]):
    while x in range(img.shape[0]):
        if y + patch_size <= img.shape[1]: 
            if x + patch_size <= img.shape[0]:
                patch_list.append(img[x:x+patch_size, y:y+patch_size, :])
        '''
            else: 
                patch_list.append(img[x:, y:y+patch_size, :])
        else:
            if x + patch_size <= img.shape[0]:
                patch_list.append(img[x:x+patch_size, y:, :])
            else: 
                patch_list.append(img[x:, y:, :])
        '''
        x += patch_size
    y += patch_size
    x = 0

In [ ]:
x = 0
y = 0

gt_list = []

while y in range(gt.shape[1]):
    while x in range(gt.shape[0]):
        if y + patch_size <= gt.shape[1]: 
            if x + patch_size <= gt.shape[0]:
                gt_list.append(gt[x:x+patch_size, y:y+patch_size])
            #else: 
                #gt_list.append(gt[x:, y:y+patch_size])
        #else:
            #if x + patch_size <= gt.shape[0]:
                #gt_list.append(gt[x:x+patch_size, y:])
            #else: 
                #gt_list.append(gt[x:, y:])
        x += patch_size
    y += patch_size
    x = 0

In [ ]:
shape_list = []

for i in range(len(gt_list)):
    shape_list.append(np.shape(gt_list[i]))

In [ ]:
gt_list[1][3,3]

In [ ]:
utils.display_dataset(patch_list[453], gt_list[453], rgb_bands, label_values, palette, vis)

In [ ]:
utils.display_predictions(convert_to_color(gt_list[453]), vis)

In [ ]:
gt_list[453][3,3]

In [ ]:
class_list = np.zeros(len(label_values))
gt_center = np.zeros(len(gt_list))

for i in range(len(gt_list)):
    c = gt_list[i][3,3]
    class_list[c] += 1
    gt_center[i] = c

In [ ]:
print(class_list)

In [ ]:
print(np.sum(class_list[1:]))

In [ ]:
np.argmax(np.bincount(gt_list[352].flatten()))

In [ ]:
class_major_list = np.zeros(len(label_values))
gt_major = np.zeros(len(gt_list))

for i in range(len(gt_list)):
    amount = np.bincount(gt_list[i].flatten())
    c = np.argmax(amount)
    class_major_list[c] += 1
    gt_major[i] = c

In [ ]:
print(class_major_list)

In [ ]:
print(np.sum(class_major_list[1:]))

In [25]:
from datasets import get_patch_data, get_pixel_idx, HyperX_patches
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import LambdaLR
import torch.optim as optim
from torch.nn import init
from torch.autograd import Variable
import torch.utils.data as data

In [16]:
from models import HamidaEtAl
import math

In [61]:
train_img, train_gt, test_img, test_gt, label_values, ignored_labels, rgb_bands, palette = get_patch_data('Salinas', 5, target_folder=data_path)


In [9]:
n_bands = train_img.shape[-1]
n_classes = len(label_values) - len(ignored_labels)

In [10]:
model = HamidaEtAl(n_bands, n_classes,
                       patch_size=5)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9,
                          nesterov=True)

In [11]:
idx_sup, idx_val, idx_unsup = get_pixel_idx(train_img, train_gt, ignored_labels, 5)


In [12]:
train_labeled_gt = [train_gt[p_l, x_l, y_l] for p_l, x_l, y_l in idx_sup]
samples_class = np.zeros(n_classes)
for c in np.unique(train_labeled_gt):
    samples_class[c-1] = np.count_nonzero(train_labeled_gt == c)

In [26]:
hyperparams = {'patch_size': 5, 'dataset': 'Salinas', 'ignored_labels': ignored_labels, 
               'flip_augmentation': True, 'radiation_augmentation': False, 'mixture_augmentation': False,
              'center_pixel': True, 'supervision': 'full'}

val_dataset = HyperX_patches(train_img, train_gt, idx_val, labeled=True, **hyperparams)
val_loader = data.DataLoader(val_dataset, batch_size=10)

train_labeled_dataset = HyperX_patches(train_img, train_gt, idx_sup, labeled=True, **hyperparams)
train_labeled_loader = data.DataLoader(train_labeled_dataset, batch_size=10,
                                       #pin_memory=True, num_workers=5,
                                       shuffle=True, drop_last=True)

unlabeled_ratio = math.ceil(len(idx_unsup)/len(idx_sup))

train_unlabeled_dataset = HyperX_patches(train_img, train_gt, idx_unsup, labeled=False, **hyperparams)
train_unlabeled_loader = data.DataLoader(train_unlabeled_dataset, batch_size=10*unlabeled_ratio,
                                       #pin_memory=True, num_workers=5,
                                       shuffle=True, drop_last=True)
amount_labeled = idx_sup.shape[0]

In [100]:
data = train_img[idx_unsup[0][0], 0:5:, 9:14]
data_aug = np.zeros_like(data)
data_train = data - np.mean(data, axis=(0,1,2))
for x,y in np.ndenumerate(data[:,:,0]):
    print(x)
    print(y)

(0, 0)
0.0010744985
(0, 1)
0.0010744985
(0, 2)
0.0010744985
(0, 3)
0.0010744985
(0, 4)
0.0010744985
(1, 0)
0.0010744985
(1, 1)
0.0010744985
(1, 2)
0.0010744985
(1, 3)
0.0010744985
(1, 4)
0.0010744985
(2, 0)
0.036891118
(2, 1)
0.045487106
(2, 2)
0.045487106
(2, 3)
0.0010744985
(2, 4)
0.0010744985
(3, 0)
0.027459407
(3, 1)
0.036055397
(3, 2)
0.036055397
(3, 3)
0.0010744985
(3, 4)
0.0010744985
(4, 0)
0.03760745
(4, 1)
0.03760745
(4, 2)
0.03760745
(4, 3)
0.0010744985
(4, 4)
0.0010744985


In [86]:
train_img[0].shape

(26, 14, 204)

In [29]:
idx_unsup[0]

array([ 0,  2, 12])

In [ ]:
train_gt, test_gt = utils.sample_gt(gt, 0.3,
                                        mode='disjoint')

train_gt, val_gt = utils.sample_gt(train_gt, 0.95, mode='disjoint')

val_dataset = HyperX(img, val_gt, labeled=True, **hyperparams)
val_loader = data.DataLoader(val_dataset,
                             batch_size=10)

train_labeled_gt, train_unlabeled_gt = utils.sample_gt(train_gt, 1/(7 + 1),
                                                        mode='disjoint')

samples_class = np.zeros(n_classes)
for c in np.unique(train_labeled_gt):
    samples_class[c-1] = np.count_nonzero(train_labeled_gt == c)

train_labeled_dataset = HyperX(img, train_labeled_gt, labeled=True, **hyperparams)
train_labeled_loader = data.DataLoader(train_labeled_dataset, batch_size=10,
                                       #pin_memory=True, num_workers=5,
                                       shuffle=True, drop_last=True)

train_unlabeled_dataset = HyperX(img, train_unlabeled_gt, labeled=False, **hyperparams)
train_unlabeled_loader = data.DataLoader(train_unlabeled_dataset,
                                         batch_size=10*7,
                                         #pin_memory=True, num_workers=5,
                                         shuffle=True, drop_last=True)

In [ ]:
samples_labeled = []

for i in range(train_labeled_dataset.__len__()):
    samples_labeled.append(train_labeled_dataset.__getitem__(i)[0])

In [ ]:
samples_labeled[0]

In [ ]:
samples_train = []

for i in range(train_dataset.__len__()):
    samples_train.append(train_dataset.__getitem__(i)[0])

In [ ]:
samples_train[0]

In [9]:
np.max(test_img)

1.1002865

In [16]:
from sklearn.metrics import classification_report

In [18]:
print(classification_report(test_gt.flatten(), test_gt.flatten()))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     64809
           1       1.00      1.00      1.00      1787
           2       1.00      1.00      1.00      3356
           3       1.00      1.00      1.00      1976
           4       1.00      1.00      1.00      1394
           5       1.00      1.00      1.00      2527
           6       1.00      1.00      1.00      3959
           7       1.00      1.00      1.00      3579
           8       1.00      1.00      1.00      9860
           9       1.00      1.00      1.00      5983
          10       1.00      1.00      1.00      2441
          11       1.00      1.00      1.00       968
          12       1.00      1.00      1.00      1903
          13       1.00      1.00      1.00       875
          14       1.00      1.00      1.00       883
          15       1.00      1.00      1.00      5958
          16       1.00      1.00      1.00      1778

    accuracy              

In [19]:
for i in range(0,5):
    print(i)

0
1
2
3
4
